# Práctica guiada (1h 40min): Regresión Lineal con *energy_consumption.csv*

**Objetivo:** construir, paso a paso, un modelo de **Regresión Lineal** para predecir consumo/energía a partir de variables explicativas.  
**Tiempo sugerido (100 min):**
- 0–10' Setup & carga de datos
- 10–35' EDA básico y limpieza mínima
- 35–60' División train/test y primer modelo lineal
- 60–80' Visualizaciones (pred vs real, residuos)
- 80–95' Iteración (mejoras sencillas)
- 95–100' Conclusiones + preguntas


## 1) Carga de datos (setup)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

CSV_PATH = r"energy_consumption.csv"  # cambia si tu ruta es distinta
df = pd.read_csv(CSV_PATH)

print("Dimensiones:", df.shape)
df.head()

## 2) Reconocimiento de columnas y configuración mínima

In [ ]:
# Intento automático de detectar la variable objetivo (y) y algunas features numéricas (X)
cols = df.columns.tolist()
lower = {c: c.lower() for c in cols}

# Candidatos de objetivo por nombre:
target_keys = ["consumption","energy","target","y","kwh","load","demand"]
target_candidates = [c for c in cols if any(k in lower[c] for k in target_keys)]

# Si no hay candidatos por nombre, toma la última columna numérica
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
TARGET = target_candidates[0] if target_candidates else (num_cols[-1] if num_cols else cols[-1])

# Elige features numéricas (evita el TARGET)
feature_candidates = [c for c in num_cols if c != TARGET]
FEATURES = feature_candidates[:6] if len(feature_candidates) >= 2 else feature_candidates

print("TARGET sugerido:", TARGET)
print("FEATURES sugeridas:", FEATURES)

# === TODO 1 ===
# Si no te convence, SOBREESCRIBE aquí manualmente:
# TARGET = "nombre_de_tu_columna_objetivo"
# FEATURES = ["col1","col2", ...]


## 3) EDA básico (10–25 min)

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

In [ ]:
# Histogramas de TARGET y de las primeras 2–3 features
cols_to_plot = [TARGET] + FEATURES[:3]
for c in cols_to_plot:
    plt.figure()
    df[c].hist(bins=20)
    plt.title(f"Histograma de {c}")
    plt.xlabel(c); plt.ylabel("Frecuencia")
    plt.show()

In [ ]:
# Dispersión (scatter) de TARGET vs primeras 2 features
for c in FEATURES[:2]:
    plt.figure()
    plt.scatter(df[c], df[TARGET])
    plt.title(f"{TARGET} vs {c}")
    plt.xlabel(c); plt.ylabel(TARGET)
    plt.show()

**Preguntas rápidas (EDA):**
- ¿Qué variable parece tener relación más clara con el objetivo?
- ¿Detectas valores atípicos evidentes (outliers)?
- ¿Hay nulos? Si los hay, ¿cómo los tratamos? (borrar filas o imputar)

In [ ]:
# === TODO 2 (opcional) ===
# Limpieza mínima de nulos
# df = df.dropna(subset=[TARGET] + FEATURES)
# for c in FEATURES:
#     if df[c].isna().any():
#         df[c] = df[c].fillna(df[c].median())
df.shape

## 4) Primer modelo: train/test y Regresión Lineal (25–40 min)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score

X = df[FEATURES].values
y = df[TARGET].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

lin = LinearRegression()
lin.fit(X_train, y_train)

y_pred = lin.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

# --- MÉTRICAS DE EVALUACIÓN ---
# MAE (Mean Absolute Error): error absoluto medio. Promedia cuánto nos equivocamos en unidades
# reales del objetivo (por ejemplo, kWh). Es fácil de interpretar y robusto ante algunos outliers.
#
# RMSE (Root Mean Squared Error): raíz del error cuadrático medio. Penaliza más los errores grandes.
# Útil cuando quieres castigar mucho los fallos grandes.
#
# R^2 (coeficiente de determinación): porcentaje de variabilidad del objetivo que explica el modelo
# respecto a un modelo tonto que siempre predice la media. 
# 1.0 es perfecto, 0.0 equivale a la media, y valores negativos indican que el modelo lo hace peor que predecir siempre la media.


print("Coeficientes (betas):", lin.coef_)
print("Intercepto:", lin.intercept_)
print("MAE:", mae)
print("RMSE:", rmse)
print("R^2:", r2)

## 5) Visualizaciones clave (20 min)

In [ ]:
# y_pred vs y_test
plt.figure()
plt.scatter(y_test, y_pred)
min_v = min(y_test.min(), y_pred.min())
max_v = max(y_test.max(), y_pred.max())
plt.plot([min_v, max_v], [min_v, max_v])
plt.title("Predicho vs Real (línea ideal y=x)")
plt.xlabel("Real"); plt.ylabel("Predicho")
plt.show()

In [ ]:
# Residuos
res = y_test - y_pred
plt.figure()
plt.scatter(y_pred, res)
plt.axhline(0)
plt.title("Residuos vs Predicción (nube sin patrón = OK)")
plt.xlabel("Predicho"); plt.ylabel("Residuo")
plt.show()

plt.figure()
plt.hist(res, bins=20)
plt.title("Histograma de residuos")
plt.xlabel("Residuo"); plt.ylabel("Frecuencia")
plt.show()

## 6) Itera y mejora (15–20 min)

In [ ]:
# === TODO 3 ===
# Prueba, UNA a la vez:
# 1) Añadir/quitar features y reentrenar. ¿Qué ocurre en el modelo?
# 2) Cambiar test_size a 0.4 y comentar cómo cambian las métricas.


### Conclusiones (escribe 5–7 bullets)
- Variables más útiles
- Métrica preferida (MAE vs RMSE) y por qué
- Patrón de residuos
- Mejora que más ayudó
- Próximos pasos


---
### Rúbrica breve (10 pts)
1) Carga + EDA (2) · 2) Modelo + métricas (3) · 3) Visualizaciones (2) · 4) Iteración documentada (2) · 5) Conclusiones claras (1)
